In [1]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from strat_tf.premodel_transformer import PreModelTransformer
from strat_tf.model_transformer import ModelTransformer
from strat_tf.modeler import Modeler
from strat_tf.sim_transformer import SimTransformer
# from transformer.predictor_transformer import PredictorTransformer
from strat_tf.preprocessor import Preprocessor
# from preprocessor.predictor_preprocessor import PredictorPreprocessor
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import math
tf.compat.v1.logging.set_verbosity(
    0
)
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [2]:
## Loading Constants
start = "2008-01-01"
end = "2020-09-01"
# Loading Databases
strat_db = Strategy("predictive_series_quarterly")
market = Market()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500")
market.close()
reload = False
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
refined = []
refined_quarterly = []
market.connect()
sec.connect()
strat_db.connect()
if reload:
    strat_db.drop_table("sim_tf")
    market.drop_table("dataset_tf")
    for i in tqdm(range(46,47)):
        try:
            ticker = tickers["Symbol"][i]
            cik = int(tickers.iloc[i]["CIK"].item())
            prices = market.retrieve_price_data("prices",ticker)
            filing = sec.retrieve_filing_data(cik)
            prices.reset_index(inplace=True)
            mt = PreModelTransformer(ticker,start,end,90)
            final = mt.fundamental_daily_merge(prices.copy(),filing.copy())
            market.store_data("dataset_tf",final)
        except Exception as e:
            print(str(e),ticker)
strat_db.close()
market.close()
sec.close()

In [10]:
prediction_days = 5
batch_size = prediction_days * 4
number_of_batches = int(100/batch_size)
training_days = batch_size * number_of_batches
sims = []
sec.connect()
strat_db.connect()
market.connect()
simulation_start = "2016-01-01"
timeline = DateUtils.create_timeline(simulation_start,end)
prices = market.retrieve_data("dataset_regression")
shift = 1
simulation = []
for i in tqdm(range(493,len(tickers["Symbol"]))):
    try:
        ticker = tickers.iloc[i]["Symbol"].replace(".","-")
        print(ticker)
        if ticker in prices.columns:
            dataset = market.retrieve_data("dataset_regression")
            pt = PreModelTransformer(ticker)
            data = pt.daily_merge(dataset,shift)
            quarter = timeline["date"].iloc[0].quarter
            quarter = 0
            m = Modeler(ticker,prediction_days)
            for date in timeline["date"]:
                try:
                    if date.weekday() < 5:
                        relevant_data = data[data["date"] <= date]
                        mr = Preprocessor(ticker)
                        if date.quarter - quarter != 0:
                            m = Modeler(ticker,prediction_days)
                            quarter = date.quarter
                            training_set = relevant_data.iloc[-(training_days+batch_size):-1]
                            refined = mr.preprocess_price_regression(training_set,ticker,batch_size,prediction_days,shift)
                            results = m.tf_model(refined)
                            for i in range(10):
                                results_2 = m.tf_model(refined)
                                if (results["results"][1] - results_2["results"][1]) < 0:
                                    break
                                if abs((results["results"][1] - results_2["results"][1]) / results["results"][1]) < 0.01:
                                    break
                                else:
                                    results = results_2
                        model = results["model"]
                        prediction_rows = relevant_data.iloc[-1:]
                        prediction_refined = mr.preprocess_prediction(prediction_rows)
                        predictions = model.predict(tf.stack([prediction_refined]))
                        packages = SimTransformer.transform(date,ticker,data,predictions,results,shift)
                        simulation.extend(packages)
                        if len(packages) > 0:
                            strat_db.store_data("sim_tf",pd.DataFrame(packages))
                except Exception as e:
                    message = {"date":date,"ticker":ticker,"message":str(e)}
                    print(message)
    except Exception as e:
        print(str(e))
sec.close()
market.close()
strat_db.close()

  0%|                                                                                                             | 0/12 [00:00<?, ?it/s]

WMB
1/1 [==============================] - 0s 2ms/step - loss: 15.8156 - mean_absolute_percentage_error: 15.8156


  8%|████████▎                                                                                           | 1/12 [05:15<57:50, 315.47s/it]

WLTW
WYNN
1/1 [==============================] - 0s 2ms/step - loss: 88.6792 - mean_absolute_percentage_error: 88.6792


1/1 [==============================] - 0s 2ms/step - loss: 13.6606 - mean_absolute_percentage_error: 13.6606


...

 42%|█████████████████████████████████████████▋                                                          | 5/12 [30:51<41:55, 359.29s/it]

XLNX
1/1 [==============================] - 0s 1ms/step - loss: 52.0838 - mean_absolute_percentage_error: 52.0838


1/1 [==============================] - 0s 1ms/step - loss: 12.5388 - mean_absolute_percentage_error: 12.5388


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [40:01<00:00, 200.15s/it]

XYL
YUM
ZBRA
ZBH
ZION
ZTS


In [8]:
timeline[timeline["date"] == datetime(2020,4,1)].index

Int64Index([1552], dtype='int64')